# 0.1.0 Load Data
* Codex Data obtained from: http://welikesharingdata.blob.core.windows.net/forshare/index.html
* Manuscript: https://www.cell.com/cell/pdf/S0092-8674(18)30904-8.pdf

In [1]:
import pandas as pd
df = {}

In [2]:
from clustergrammer2 import net

>> clustergrammer2 backend version 0.4.2


In [3]:
import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot

In [4]:
from copy import deepcopy

In [5]:
from glob import glob

### Load X-shift cluster ID

In [6]:
df['cell_type'] = pd.read_excel('../data/big_data/mmc2__codex_cell_type_info.xlsx', index_col=0)
df['cell_type'].shape

(58, 1)

In [7]:
ct_dict = {}
cell_types = []
for inst_id in df['cell_type'].index.tolist():
    inst_ct = df['cell_type']['Imaging phenotype (cell type)'].loc[inst_id]
    cell_types.append(inst_ct)
    ct_dict[inst_id] = inst_ct

In [8]:
cell_types = sorted(list(set(cell_types)))
print(len(cell_types))
cell_types

28


['B cells',
 'B220(+) DN T cells',
 'CD106(+)CD16/32(+)CD31(+) stroma',
 'CD106(+)CD16/32(+)CD31(-)Ly6C(-) stroma',
 'CD106(+)CD16/32(-)Ly6C(+)CD31(+)',
 'CD106(-)CD16/32(+)Ly6C(+)CD31(-)',
 'CD106(-)CD16/32(-)Ly6C(+)CD31(+) stroma',
 'CD11c(+) B cells',
 'CD3(+) other markers (-)',
 'CD31(hi) vascular',
 'CD4(+) T cells',
 'CD4(+)CD8(-)cDC',
 'CD4(+)MHCII(+)',
 'CD4(-)CD8(+)cDC',
 'CD4(-)CD8(-) cDC',
 'CD8(+) T cells',
 'ERTR7(+) stroma',
 'F4/80(+) mphs',
 'FDCs',
 'NK cells',
 'capsule',
 'dirt',
 'erythroblasts',
 'granulocytes',
 'marginal zone mphs',
 'megakaryocytes',
 'noid',
 'plasma cells']

In [9]:
x_dim = 1200
y_dim = 1000

## Check available Files

In [10]:
glob('../data/big_data/*.txt')

['../data/big_data/channelNames_BALBcMRLdataset.txt']

In [11]:
glob('../data/big_data/*.csv')

['../data/big_data/CODEX_MRLdataset_neighborhood_graph.csv',
 '../data/big_data/BALBcMRLdataset_exposuretimes.csv',
 '../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv']

### Expression Data
See http://welikesharingdata.blob.core.windows.net/forshare/index.html

* CSV file contains mean marker intensities and X, Y, Z corrdinates for each cell relative to the top left corner of its tile. 
* Tile IDs correspond to tile images sorted in the alphanumeric order
* Size parameter corresponds to the cell object sizes in voxels
* "imaging phenotype Cluster ID" column specifies the phenotypic cluster identity as determined by X-shift
* "niche Cluster ID" column specifies the i-niche cluster determined by K-means (K=100)
* neighborhood graph files describes which pairs of cells are adjacent to one another

### Add X-Y position ID to the Cells
This ID will be used to generate the neighborhood graph.

In [12]:
all_tiles = ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01', 'BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']

In [13]:
%%time
df['exp'] = pd.read_csv('../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')

CPU times: user 9.66 s, sys: 659 ms, total: 10.3 s
Wall time: 10.1 s


In [14]:
df['exp'].shape

(734101, 38)

In [ ]:
%%time
# new_rows = ['C-' + str(x) for x in df['exp'].index.tolist()]

new_rows = []
for inst_row in df['exp'].index.tolist():
    inst_ser = df['exp'].loc[inst_row]
    inst_x = inst_ser['X.X']
    inst_y = inst_ser['Y.Y']
    inst_tile = inst_ser['sample_Xtile_Ytile']
    
    new_row = 'C-' + str(inst_row) + '_' + inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    # new_row = inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    new_rows.append(new_row)

df['exp'].index = new_rows
print(df['exp'].shape)

In [ ]:
rows = df['exp'].index.tolist()
print(len(rows))
print(len(list(set(rows))))

In [ ]:
df['exp'].index.tolist()[0]

In [ ]:
new_row

In [ ]:
df['exp'].head()

In [ ]:
df['exp'].shape

### Add Sample Column

In [ ]:
sample_list = [x.split('_')[0] for x in list(df['exp']['sample_Xtile_Ytile'].get_values())]
ser_sample = pd.Series(sample_list, name='sample_slide', index=df['exp'].index.tolist())

In [ ]:
print(len(sorted(list(set(sample_list)))))
list_slides = sorted(list(set(sample_list)))
list_slides

In [ ]:
df['exp']['sample_slide'] = ser_sample

In [ ]:
ser_sample.value_counts()

In [ ]:
cols = df['exp'].columns.tolist()
cols

In [ ]:
exp_cols = cols[1:30]
exp_cols

In [ ]:
unique_dict = {}
for inst_col in cols:
    inst_list_unique = list(df['exp'][inst_col].unique())
    unique_dict[inst_col] = inst_list_unique    
    inst_num_unique = len(inst_list_unique)
    print(inst_col, inst_num_unique)

### Select Single Image Tile
BALBc: normal tissue
MRL/lpr: spleen from animals with systemic autoimmune disease

Start with: 'BALBc-1_X01_Y01'

In [ ]:
df_list = []


for inst_tile in all_tiles:
# for inst_tile in ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01']:
# for inst_tile in ['BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']:    
    keep_rows = []

    ser_tile = df['exp']['sample_Xtile_Ytile']
    ser_found = ser_tile[ser_tile == inst_tile]
    
    keep_rows.extend(ser_found.index.tolist())
    inst_df = df['exp'].loc[keep_rows].transpose()
    print('inst_df', inst_df.shape)
    
    if 'X02' in inst_tile:
        inst_df.loc['X.X'] = inst_df.loc['X.X'] + 1350

    if 'Y01' in inst_tile:        
        inst_df.loc['Y.Y'] = 2000 - inst_df.loc['Y.Y']
    
    if 'Y02' in inst_tile:
        inst_df.loc['Y.Y'] = 1000 - inst_df.loc['Y.Y'] 
        
    df_list.append(inst_df)
    
    
# df['tile'] = df['exp'].loc[keep_rows].transpose()
df['tile'] = pd.concat(df_list, axis=1)
print(df['tile'].shape)

cats = df['tile'].loc['Imaging phenotype cluster ID']
cats = [ct_dict[x] for x in cats]

new_cols = []

cols = df['tile'].columns.tolist()
for index in range(len(cols)):
    
    new_col = (cols[index], 'Cell Type: ' + str(cats[index]))
    new_cols.append(new_col)
df['tile'].columns = new_cols
df['tile'].shape

In [ ]:
df['tile'].columns.tolist()[0]

### Plot Tile Expression Levels

In [ ]:
df['tile-exp-ini'] = df['tile'].loc[exp_cols]
df['tile-exp-ini'].shape

### Set Negative Expression Levels to Zero

In [ ]:
df['tile-exp'] = df['tile-exp-ini']
df['tile-exp'][df['tile-exp'] < 0] = 0

In [ ]:
df['tile-exp'].transpose().describe()

In [ ]:
df['tile-exp'][df['tile-exp'] > 5000] = 5000

In [ ]:
df['tile-exp'].shape

In [ ]:
df['tile'].loc['Z.Z'].head()

### Location of Cells

In [ ]:
df['tile-loc'] = df['tile'].loc[['X.X', 'Y.Y']].transpose()
df['tile-loc'].shape

In [ ]:
# df['tile-loc']['Y.Y'] = 1000 - df['tile-loc']['Y.Y'] 

In [ ]:
df['tile-loc'] = df['tile-loc'].astype('int')

In [ ]:
def set_expression_opacity(inst_gene):
    ser_opacity = df['tile-exp'].loc[inst_gene]
    list_opacity = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
    scatter.default_opacities = list_opacity

In [ ]:
len(df['tile-loc'].index.tolist())

In [ ]:
fig = plt.figure(title='Scatter')
def_tt = bqplot.Tooltip(fields=['name'], formats=[''])

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'], 
                      figsize=(20,10), 
                      ylim=(0,1000), 
                      xlim=(0,1000), stroke='black', 
                      tooltip=def_tt, 
                      names=df['tile-loc'].index.tolist(), 
                      display_names=False)

inst_width = 900
fig.layout.min_height = str(inst_width/1.2) + 'px'
fig.layout.min_width = str(inst_width) + 'px'

set_expression_opacity('NKp46')

scatter.default_size = 100
scatter.colors = ['red']

### Using RGB to HEX
https://www.rgbtohex.net/

In [ ]:
cat_colors = {}
# A 
cat_colors['NK cells'] = '#FB0006'
# B
cat_colors['granulocytes'] = '#FA1400'
# C 
cat_colors['CD4(-)CD8(-) cDC'] = '#FC4B08'
# D 
cat_colors['B220(+) DN T cells'] = '#FD8007'
# E 
cat_colors['plasma cells'] = '#FDBA0A'
# F 
cat_colors['F4/80(+) mphs'] = '#FFF80B'
# G 
cat_colors['FDCs'] = '#FC9CA0'
# H
cat_colors['CD11c(+) B cells'] = '#99FF06'
# I 
cat_colors['capsule'] = '#68FF0A'
# J 
cat_colors['marginal zone mphs'] = '#0A4600'
# K 
cat_colors['noid'] = '#25FF04'
# L 
cat_colors['B cells'] = '#FFFF09'
# M
cat_colors['erythroblasts'] = '#1FFF3C'
# N
cat_colors['CD106(+)CD16/32(+)CD31(+) stroma'] = '#23FF6D'
# O
cat_colors['CD4(-)CD8(+)cDC'] = '#23FFA3'
# P 
cat_colors['CD106(-)CD16/32(-)Ly6C(+)CD31(+) stroma'] = '#20FFDD'
# Q
cat_colors['megakaryocytes'] = '#1CE5FF'
# R 
cat_colors['CD106(-)CD16/32(+)Ly6C(+)CD31(-)'] = '#15A7FF'
# S
cat_colors['CD4(+) T cells'] = '#0D6FFF'
# T
cat_colors['CD4(+)MHCII(+)'] = '#0137FF'
# Ud
cat_colors['CD31(hi) vascular'] = '#0000FF'
# V
cat_colors['CD3(+) other markers (-)'] = '#0700FF'
# W
cat_colors['CD106(+)CD16/32(+)CD31(-)Ly6C(-) stroma'] = '#2D00FF'
# X
cat_colors['CD8(+) T cells'] = '#5900FF'
# Y
cat_colors['ERTR7(+) stroma'] = '#8C00FE'
# Z
cat_colors['CD106(+)CD16/32(-)Ly6C(+)CD31(+)'] = '#C300FF'
# [
cat_colors['CD4(+)CD8(-)cDC'] = '#FB00FA'
# dirt
cat_colors['dirt'] = '#BCBCBC'

### Expression Levels

In [ ]:
keep_cols = [x for x in df['tile-exp'].columns.tolist() if 'dirt' not in x[1]]
df['tile-exp-clean'] = df['tile-exp'][keep_cols]
df['tile-exp-clean'].shape

In [ ]:
net.load_df(df['tile-exp-clean'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=1, cat_title='Cell Type')

In [ ]:
net.load_df(df['tile-exp-clean'])
net.widget()

### Voronoi Plot

In [ ]:
from scipy.spatial import Voronoi
vor = Voronoi(df['tile-loc'])

In [ ]:
point_list = df['tile-loc'].index.tolist()
point_names = [x[0] for x in point_list]
cat_names   = [x[1].split(': ')[1] for x in point_list]

In [ ]:
patch_data = {}
patch_data['x'] = []
patch_data['y'] = []
patch_data['colors'] = []
region_labels = []

region_point_dict = {}
for point_index in range(vor.point_region.shape[0]):
    region_index = vor.point_region[point_index]
    region_point_dict[region_index] = point_index

for region_index in range(len(vor.regions)):
    
    inst_region = vor.regions[region_index]

    if -1 not in inst_region and len(inst_region) > 0:

        point_index = region_point_dict[region_index]
        point_cat = cat_names[point_index]
        region_labels.append(point_cat)
        
        # save cat_colors
        inst_color = cat_colors[point_cat]
        patch_data['colors'].append(inst_color)
        
        x_list = []
        y_list = []
        for inst_vertex in inst_region:
            inst_pos = vor.vertices[inst_vertex]
            x_list.append(inst_pos[0])
            y_list.append(inst_pos[1])
            
        patch_data['x'].append(x_list)
        patch_data['y'].append(y_list)    

In [ ]:
import bqplot.pyplot as plt
fig = plt.figure(animation_duration=1000)
patch = plt.plot([], [], 
                 fill='inside',
                 fill_colors=patch_data['colors'],
                 stroke_width=1,
                 close_path=True,
                 labels=region_labels,
                 tooltip=def_tt,
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                )

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'],
#                       tooltip=def_tt, 
                      names=point_names,
                      display_names=False, default_size=2)


inst_width = 950
fig.layout.min_height = str(inst_width/(1.15)) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

patch.x = patch_data['x']
patch.y = patch_data['y']

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)

fig
# plt.show()

In [ ]:
def mouseover_highlight(self, target):
    # print('cat name', target['data']['name'])
    list_opacities = []
    for inst_label in region_labels:
        inst_opacity = 0.25
        if inst_label == target['data']['name']:
            inst_opacity = 1
        list_opacities.append(inst_opacity)

    self.opacities = list_opacities

In [ ]:
def reset_highlight(self, target):
#     print('CLICKING')
    list_opacities = [1 for x in region_labels]
    self.opacities = list_opacities

In [ ]:
patch.on_hover(mouseover_highlight)
patch.on_element_click(reset_highlight)

In [ ]:
cells = df['tile-loc'].index.tolist()
len(cells)

In [ ]:
df['tile-loc'].transpose()

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
mat_cd = pdist(df['tile-loc'], metric='euclidean')

In [ ]:
dist_mat = squareform(mat_cd)
dist_mat.shape

In [ ]:
df['dist'] = pd.DataFrame(data=dist_mat, index=cells, columns=cells)

In [ ]:
df['dist'].head()

In [ ]:
# df['dist'][inst_col].hist()

In [ ]:
# ser_hood = inst_ser[inst_ser < 100]

In [ ]:
# ser_hoodod.shape

In [ ]:
num_nn = 10
list_cell_density = []
list_nn_dist = []
for inst_col in df['dist'].columns.tolist():
    # skip self distance of zero
    inst_ser = df['dist'][inst_col].sort_values()[1:]
    
    ser_nn = inst_ser[:num_nn]
    
    # find nearest neighbor cell type distribution
    nn_names = ser_nn.index.tolist()
    nn_cats = [x[1].split(': ')[1] for x in nn_names]
    nn_dist = pd.Series(nn_cats).value_counts()/num_nn
    
    # number of cells in 100px neighborhood
    cell_density = inst_ser[inst_ser < 100].shape[0]
    list_cell_density.append(cell_density)

    nn_dist.name = (inst_col[0], inst_col[1], 'NN CT: ' + nn_dist.index.tolist()[0], 'Density: ' + str(cell_density))
    list_nn_dist.append(nn_dist)

df['nn-dist'] = pd.concat(list_nn_dist, axis=1, sort=True).fillna(0)    

In [ ]:
# nn_dist

In [ ]:
net.load_df(df['nn-dist'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=2, cat_title='NN CT')

# Neighborhood Graph

In [ ]:
df['exp'].index.tolist()[0]

In [ ]:
df['neighbor'] = pd.read_csv('../data/big_data/CODEX_MRLdataset_neighborhood_graph.csv')
df['neighbor'].shape

In [ ]:
df['neighbor'].head()

In [ ]:
all_tiles = ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01', 'BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']

### Make Dictionary of Nieghbor Cells and Neighbor Cell Types

In [ ]:
%%time
neighbor_graph = {}
ct_graph = {}
for inst_row in df['neighbor'].index.tolist():
    inst_ser = df['neighbor'].loc[inst_row]
    
    inst_tile = inst_ser['Cell1 Sample_Xtile_Ytile']
    if inst_tile in all_tiles:
        inst_cell = inst_tile + '_' + str(inst_ser['X1']) + '_' + str(inst_ser['Y1'])
    
        inst_neighbor = inst_ser['Cell2 Sample_Xtile_Ytile'] + '_' + str(inst_ser['X2']) + '_' + str(inst_ser['Y2'])
        inst_cat = ct_dict[inst_ser['Cell2 Imaging phenotype cluster ID']]
        
        # add cell to dictionary if necessary
        if inst_cell not in neighbor_graph:
            neighbor_graph[inst_cell] = []
            ct_graph[inst_cell] = []
            
        neighbor_graph[inst_cell].append(inst_neighbor)
        ct_graph[inst_cell].append(inst_cat)

In [ ]:
cell_keys = list(neighbor_graph.keys())
print(len(cell_keys))
print(len(list(set(cell_keys))))

In [ ]:
df['exp'].index.tolist()[0]

In [ ]:
list(neighbor_graph.keys())[0]

In [ ]:
'_'.join(['a', 'b'])

In [ ]:
list(neighbor_graph.keys())[0]

In [ ]:
df['tile'].columns.tolist()[0]

### Loading neighbor graph for cells in tile

In [ ]:
common_cat_neighbor = {}
second_cat_neighbor = {}
number_neighbors = {}
other_neighbor = {}
for inst_col in df['tile'].columns.tolist():
    inst_name = inst_col[0]
    inst_name = '_'.join(inst_name.split('_')[1:])

    found_neighbors = neighbor_graph[inst_name]
    found_cts = ct_graph[inst_name]
    
    number_neighbors[inst_col[0]] = len(found_cts)
    
    ser_cats = pd.Series(found_cts).value_counts()
    rank_cats = ser_cats.index.tolist()
    
    # find other cat neighbor
    found_it = False
    col_cat = inst_col[1].split(': ')[1]
    other_cat_neighbor = rank_cats[0]
    for inst_cat in rank_cats:
        if inst_cat != col_cat and found_it == False:
            other_cat_neighbor = inst_cat
            found_it = True
            
    other_neighbor[inst_col[0]] = other_cat_neighbor
    
    # find most common cat neighbor
    common_cat_neighbor[inst_col[0]] = rank_cats[0]
    
    if len(ser_cats.index.tolist()) == 1:
        second_cat_neighbor[inst_col[0]] = rank_cats[0]
    else:
        second_cat_neighbor[inst_col[0]] = rank_cats[1]

In [ ]:
found_neighbors

In [ ]:
list(common_cat_neighbor.keys())[0]

In [ ]:
common_cat_neighbor['C-643_BALBc-1_X01_Y01_10_277']

### Add Category for Most Common Neighbor Cat

In [ ]:
df['tile-neighbor'] = deepcopy(df['tile-exp-clean'])

In [ ]:
df['tile-neighbor'].columns.tolist()[0]

In [ ]:
new_cols = [(x[0], x[1], 
             'Neighbor: ' + other_neighbor[x[0]], 
              density_dict[x[0]]) for x in df['tile-exp-clean'].columns]
df['tile-neighbor'].columns = new_cols

In [ ]:
df['tile-neighbor'].columns.tolist()[0]

In [ ]:
net.load_df(df['tile-neighbor'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=2, cat_title='Neighbor')

# Most Common Neighborhood Cat

In [ ]:
# net.load_df(df['tile-neighbor'])
# net.widget()

# Most Common Neighborhood Cat: B Cells

In [ ]:
keep_cols = [x for x in df['tile-neighbor'].columns.tolist() if x[1].split(': ')[1] == 'B cells']
len(keep_cols)

In [ ]:
net.load_df(df['tile-neighbor'][keep_cols])
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

In [ ]:
# keep_cols = [x for x in df['tile-neighbor'].columns.tolist() if x[1].split(': ')[1] == 'CD4(+) T cells']
keep_cols = [x for x in df['tile-neighbor'].columns.tolist() if x[1].split(': ')[1] == 'F4/80(+) mphs']
len(keep_cols)

In [ ]:
net.load_df(df['tile-neighbor'][keep_cols])
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

### All Cells Neighbor Distribution

In [ ]:
net.load_df(df['nn-dist'])
net.widget()

# Merge Expression Levels and Neighborhood Distributions

In [ ]:
df['nn-dist'].shape

In [ ]:
df['nn-dist'].columns.tolist()[0]

In [ ]:
list(density_dict.keys())[0]

In [ ]:
density_dict = {}
nct_dict = {}
for inst_col in df['nn-dist'].columns.tolist():
    nct_dict[inst_col[0]] = inst_col[2]
    density_dict[inst_col[0]] = inst_col[3]

In [ ]:
df['tile-exp-clean'].shape

### Add Cell Density to Expression Levels

In [ ]:
df['exp-common'] = deepcopy(df['tile-exp-clean'])
exp_cols = [(x[0], x[1], nct_dict[x[0]], density_dict[x[0]]) for x in df['exp-common'].columns]
df['exp-common'].columns = exp_cols
df['exp-common'].index = [(x, 'Type: Expression') for x in df['exp-common'].index]
df['exp-common'].shape

### Find Common Cells

In [ ]:
df['nn-dist-common'] = df['nn-dist'][exp_cols]
df['nn-dist-common'].index = [(x, 'Type: Neighborhood') for x in df['nn-dist-common'].index]
df['nn-dist-common'].shape

In [ ]:
df['merge'] = pd.concat([df['exp-common'], df['nn-dist-common']], axis=0)
df['merge'].shape

In [ ]:
df['merge'].columns.tolist()[0]

# Merge Dist and Expression 

In [ ]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'B cells']
len(keep_cols)

In [ ]:
net.load_df(df['exp-common'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

In [ ]:
net.load_df(df['merge'][keep_cols])
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

# 'CD4(+) T cells'

In [ ]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'CD4(+) T cells']
len(keep_cols)

In [ ]:
net.load_df(df['exp-common'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

In [ ]:
net.load_df(df['merge'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

# Marginal zone mphs

In [ ]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'marginal zone mphs']
len(keep_cols)

In [ ]:
net.load_df(df['merge'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

In [ ]:
ser_density = pd.Series(list_cell_density)
ser_density.hist()

In [ ]:
nn_names = ser_nn.index.tolist()
nn_cats = [x[1].split(': ')[1] for x in nn_names]
nn_dist = pd.Series(nn_cats).value_counts()/num_nn
nn_dist

In [ ]:
inst_ser.head()

In [ ]:
inst_ser[1:].head()

In [ ]:
print(cells[0])
df['dist'][cells[0]].sort_values()